In [1]:
import sys
sys.path.insert(1, '../')
from utils.packages import *

In [2]:
#train, val, test = train_test_val_split(dataset, train_ratio=0.6, validation_ratio=0.2, test_ratio=0.2)
path='../data/datasets/synthea/'
#train.to_parquet(path + 'train')
#val.to_parquet(path + 'val')
#test.to_parquet(path + 'test')

train = pd.read_parquet(path + 'train')
val = pd.read_parquet(path + 'val')
test = pd.read_parquet(path + 'test')

In [3]:
global_params = {
    'max_seq_len': 32,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 32,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [4]:
#diagnos_codes = dataset['diagnos_code'].tolist()
#file_path = '../data/vocabularies/Synthea/labels'
#write_codes_to_file(diagnos_codes, file_path)

In [5]:
#file_path = '../data/vocabularies/Synthea/labels.npy'
#s = np.load(file_path)

In [6]:
files = {'code':'../data/vocabularies/Synthea/snomedrxnorm.npy',
         'age':'../data/vocabularies/Synthea/age.npy',
         'labels':'../data/vocabularies/Synthea/labels.npy'
        }
tokenizer = EHRTokenizer(task='nextvisit', filenames=files)

In [7]:
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 4, # number of multi-head attention layers required
    'num_attention_heads': 6, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
    'use_prior':True,
}

In [8]:
conf = BertConfig(model_config)
tensorboarddir = '../logs/'

In [9]:
condfiles = {'dd':'../data/train_stats/dd_cond_probs.empirical.p', 
             'dp':'../data/train_stats/dp_cond_probs.empirical.p', 
             'pp':'../data/train_stats/pp_cond_probs.empirical.p' , 
             'pd':'../data/train_stats/pd_cond_probs.empirical.p'
            }

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(tokenizer.getVoc('label').values()))
mlb.fit([[each] for each in list(tokenizer.getVoc('label').values())])

MultiLabelBinarizer(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                             15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                             28, 29, ...])

In [11]:
traind = EHRDatasetCodePrediction(train, max_len=train_params['max_len_seq'], conditional_files=condfiles, tokenizer=tokenizer, run_type='train') 
vald = EHRDatasetCodePrediction(val, max_len=train_params['max_len_seq'], conditional_files=condfiles,tokenizer=tokenizer, run_type='val')
testd = EHRDatasetCodePrediction(test, max_len=train_params['max_len_seq'], conditional_files=condfiles,tokenizer=tokenizer, run_type='test') 

Loading data
Loading data
Loading data


In [12]:
trainloader = torch.utils.data.DataLoader(traind, batch_size=32, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(vald, batch_size=32, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testd, batch_size=32, shuffle=True, num_workers=4)

In [13]:
trainer = pl.Trainer(
        max_epochs=5, 
        gpus=-1,
        logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
        callbacks=[pl.callbacks.progress.TQDMProgressBar()], 
        progress_bar_refresh_rate=1,
        weights_summary=None, # Can be None, top or full
        num_sanity_val_steps=10,
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [14]:
model = BertMultiLabelPrediction(conf, num_labels=len(tokenizer.getVoc('label').keys())) 
PATH = "../saved_models/MLM/model_with_prior_82test"
model = load_model(PATH, model)
params = list(model.named_parameters())
optim = adam(params, optim_param)

t_total value of -1 results in schedule not being applied


In [15]:
patienttrajectory = TrainerCodes(model, optim, optim_param, binarizer=mlb)

trainer.fit(
    patienttrajectory, 
    train_dataloaders=trainloader,
    val_dataloaders=valloader,
);

predictions = trainer.predict(patienttrajectory, dataloaders=testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 136it [00:00, ?it/s]

Process Process-10:
Exception ignored in: <function _releaseLock at 0x7f6357ee3830>
Traceback (most recent call last):
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/multiprocessing/process.py", line 290, in _bootstrap
    util._run_after_forkers()
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/multiprocessing/util.py", line 167, in _run_after_forkers
    func(obj)
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/multiprocessing/queues.py", line 76, in _after_fork
    self._send_bytes = self._writer.send_bytes
KeyboardInterrupt
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f620ab1b560>
Traceback (most recent call last):
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/dat

In [17]:
aucpr = 0
for pred in predictions:
    aucpr+=pred['AUCPR']
aucpr / len(predictions)

0.4741216475517144

In [19]:
auc = 0
for pred in predictions:
    auc+=pred['AUC']
auc / len(predictions)

0.8711814074197308